# Clustering Exploration on Word Embeddings
Applications of spectral clustering, locally linear embeddings, laplacian eigenmaps, and IsoMap on newsgroup dataset consisting of pretrained word embeddings (`Alibaba/gte-base-en` model).

In [1]:
# Read npy files from News dataset directory
import numpy as np
import os

# Define the directory where the npy file is stored
directory = '../../News'

train_embeddings = np.load(os.path.abspath(os.path.join(directory, "train_embeddings.npy")))
train_labels = np.load(os.path.join(directory, "train_labels.npy"))
test_embeddings = np.load(os.path.join(directory, "test_embeddings.npy"))
test_labels = np.load(os.path.join(directory, "test_labels.npy"))
label_embeddings = np.load(os.path.join(directory, "label_embeddings.npy"))

assert len(train_embeddings) == len(train_labels)
assert len(test_embeddings) == len(test_labels)

train_embeddings_norm = train_embeddings / np.linalg.norm(train_embeddings, axis=1, keepdims=True)
test_embeddings_norm = test_embeddings / np.linalg.norm(test_embeddings, axis=1, keepdims=True)
label_embeddings_norm = label_embeddings / np.linalg.norm(label_embeddings, axis=1, keepdims=True)

In [2]:
from datasets import load_dataset

news_dataset = load_dataset("SetFit/20_newsgroups")

assert len(set(news_dataset['train']['label_text'])) == len(set(train_labels))
assert len(set(news_dataset['test']['label_text'])) == len(set(test_labels))

print(f"Total unique train labels: {len(set(news_dataset['train']['label_text']))}")

train_labels_text = list(set(news_dataset['train']['label_text']))
for _t in train_labels_text:
    print(_t)

test_labels_text = list(set(news_dataset['test']['label_text']))
# for _t in test_labels_text:
#     print(_t)

/opt/homebrew/anaconda3/envs/311_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/anaconda3/envs/311_env/lib/python3.11/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Total unique train labels: 20
rec.autos
rec.sport.baseball
sci.space
sci.med
rec.motorcycles
comp.graphics
talk.politics.misc
soc.religion.christian
talk.politics.mideast
sci.crypt
misc.forsale
comp.sys.mac.hardware
comp.windows.x
sci.electronics
comp.sys.ibm.pc.hardware
alt.atheism
talk.politics.guns
comp.os.ms-windows.misc
rec.sport.hockey
talk.religion.misc


# Embedding Refinement Using Clusters

In [3]:
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import pairwise_distances
from tqdm import tqdm

def compute_attractors(embeddings, labels):
    unique_labels = np.unique(labels)
    attractors = {label: np.median(embeddings[labels == label], axis=0) for label in unique_labels}
    return attractors

def compute_repellers(embeddings, labels, attractors):
    repellers = {}
    unique_labels = np.unique(labels)
    for label in unique_labels:
        other_class_embeddings = embeddings[labels != label]
        other_class_centroid = np.median(other_class_embeddings, axis=0)
        repellers[label] = 2 * other_class_centroid - attractors[label]
    return repellers

def refine_embeddings(embeddings, labels, attractors, repellers, learning_rate=1e-2, num_iterations=10):
    refined_embeddings = embeddings.copy()
    momentum = np.zeros_like(embeddings)
    alpha = 0.9  # Momentum factor

    for _ in tqdm(range(num_iterations)):
        for i in range(len(refined_embeddings)):
            label = labels[i]
            attractor = attractors[label]
            repeller = repellers[label]
            
            distance_to_attractor = pairwise_distances(refined_embeddings[i:i+1], attractor.reshape(1, -1), metric='cosine')
            distance_to_repeller = pairwise_distances(refined_embeddings[i:i+1], repeller.reshape(1, -1), metric='cosine')
            
            # Update rule using attraction and repulsion with momentum
            update = learning_rate * ((attractor - refined_embeddings[i]) * distance_to_attractor +
                                      (refined_embeddings[i] - repeller) * distance_to_repeller)
            momentum[i] = alpha * momentum[i] + update
            refined_embeddings[i] += momentum[i]

    return normalize(refined_embeddings, axis=1)


attractors = compute_attractors(train_embeddings_norm, train_labels)
repellers = compute_repellers(train_embeddings_norm, train_labels, attractors)
refined_train_embeddings = refine_embeddings(train_embeddings_norm, train_labels, attractors, repellers)

100%|██████████| 10/10 [00:25<00:00,  2.50s/it]


## Using Wasserstein Barycenters

In [4]:
import numpy as np
import ot  # Import Python Optimal Transport library
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

def compute_wasserstein_barycenter(embeddings, reg=1e-3):
    embeddings = np.array(embeddings)  
    A = embeddings.T  # Transpose embeddings so that each column is a distribution
    dim, n_hists = A.shape  

    # Create a cost matrix using the squared Euclidean distance
    M = ot.dist(A, A, metric='sqeuclidean')
    M /= M.max()  # Normalize for stability
    
    weights = np.ones(n_hists) / n_hists  # Initialize weights uniformly

    # Compute the barycenter using the Sinkhorn-Stabilized algorithm
    barycenter = ot.bregman.barycenter(A, M, reg, weights=weights, method='sinkhorn_stabilized', numItermax=10000, stopThr=0.0001, verbose=False)

    if np.any(np.isnan(barycenter)):
        print("NaN detected in barycenter, attempting to recover with minimum regularization.")
        barycenter = ot.bregman.barycenter(A, M, reg + 0.01, weights=weights, method='sinkhorn_stabilized', numItermax=10000, stopThr=0.0001, verbose=False)
    
    return barycenter.reshape(1, -1)

def compute_attractors(embeddings, labels):
    unique_labels = np.unique(labels)
    attractors = {}
    for label in unique_labels:
        label_embeddings = embeddings[labels == label]
        attractor = compute_wasserstein_barycenter(label_embeddings)
        attractors[label] = attractor
    return attractors

def compute_repellers(embeddings, labels, attractors):
    repellers = {}
    unique_labels = np.unique(labels)
    for label in unique_labels:
        other_class_embeddings = embeddings[labels != label]
        other_class_centroid = np.median(other_class_embeddings, axis=0)
        repellers[label] = 2 * other_class_centroid - attractors[label]
    return repellers

def refine_embeddings(embeddings, labels, attractors, repellers, learning_rate=1e-2, num_iterations=10):
    refined_embeddings = embeddings.copy()
    momentum = np.zeros_like(embeddings)
    alpha = 0.9  # Momentum factor

    for _ in tqdm(range(num_iterations)):
        for i in range(len(refined_embeddings)):
            label = labels[i]
            attractor = attractors[label]
            repeller = repellers[label]
            
            distance_to_attractor = pairwise_distances(refined_embeddings[i:i+1], attractor.reshape(1, -1), metric='cosine')
            distance_to_repeller = pairwise_distances(refined_embeddings[i:i+1], repeller.reshape(1, -1), metric='cosine')
            
            # Update rule using attraction and repulsion with momentum
            update = learning_rate * ((attractor - refined_embeddings[i]) * distance_to_attractor +
                                      (refined_embeddings[i] - repeller) * distance_to_repeller)
            momentum[i] = alpha * momentum[i] + update
            refined_embeddings[i] += momentum[i]

    return normalize(refined_embeddings, axis=1)

attractors = compute_attractors(train_embeddings_norm, train_labels)
repellers = compute_repellers(train_embeddings_norm, train_labels, attractors)

refined_train_embeddings_bary = normalize(refine_embeddings(train_embeddings_norm, train_labels, attractors, repellers), axis=1)

/opt/homebrew/anaconda3/envs/311_env/lib/python3.11/site-packages/ot/backend.py:1143: RuntimeWarning: invalid value encountered in log
  return np.log(a)
/opt/homebrew/anaconda3/envs/311_env/lib/python3.11/site-packages/ot/bregman/_barycenter.py:513: UserWarning: Numerical errors at iteration 0
  warnings.warn('Numerical errors at iteration %s' % ii)
100%|██████████| 10/10 [00:25<00:00,  2.50s/it]


# Refine Embeddings to Be Closer to Centroids/Labels

In [5]:
import numpy as np
from sklearn.preprocessing import normalize

def modify_embeddings_with_labels(embeddings, labels, alpha=0.5):
    unique_labels = np.unique(labels)
    centroids = {}

    # Calculate the centroid for each class
    for label in unique_labels:
        centroids[label] = np.median(embeddings[labels == label], axis=0)

    # Modify embeddings to be closer to their respective centroids
    modified_embeddings = embeddings.copy()
    for i, emb in enumerate(embeddings):
        label = labels[i]
        centroid = centroids[label]

        # Move the embedding towards the centroid
        modified_embeddings[i] = alpha * centroid + (1 - alpha) * emb

    # Normalize the embeddings to ensure they are unit vectors
    modified_embeddings = normalize(modified_embeddings)

    return modified_embeddings

def modify_embeddings_with_label_embeddings(_embeddings, _labels, _label_embeddings, alpha=0.5):
    modified_embeddings = _embeddings.copy()
    for i, emb in enumerate(_embeddings):
        label = _labels[i]
        label_embedding = _label_embeddings[label]

        # Move the embedding towards the label_embedding
        modified_embeddings[i] = alpha * label_embedding + (1 - alpha) * emb

    # Normalize the embeddings to ensure they are unit vectors
    modified_embeddings = normalize(modified_embeddings)

    return modified_embeddings

train_embeddings_modified = modify_embeddings_with_labels(train_embeddings_norm, train_labels, alpha=0.5)
train_embeddings_modified_label_embeddings = modify_embeddings_with_label_embeddings(train_embeddings_norm, train_labels, label_embeddings_norm, alpha=0.5)

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

def eval_knn(_train_embeddings, _train_labels, _test_embeddings, _test_labels, k=20):
    knn_classifier = KNeighborsClassifier(n_neighbors=k, metric='cosine')
    knn_classifier.fit(_train_embeddings, _train_labels)

    y_pred = knn_classifier.predict(_test_embeddings)
    accuracy = accuracy_score(_test_labels, y_pred)
    print(f"Accuracy of kNN (k = {k}): {accuracy:.4f}")

In [7]:
print("Base KNN Accuracies")
for k in [10, 20, 30, 50, 100, 200, 300, 500]:
    eval_knn(train_embeddings_norm, train_labels, test_embeddings_norm, test_labels, k=k)

Base KNN Accuracies
Accuracy of kNN (k = 10): 0.7138
Accuracy of kNN (k = 20): 0.7196
Accuracy of kNN (k = 30): 0.7152
Accuracy of kNN (k = 50): 0.7146
Accuracy of kNN (k = 100): 0.7081
Accuracy of kNN (k = 200): 0.7035
Accuracy of kNN (k = 300): 0.6955
Accuracy of kNN (k = 500): 0.6856


In [8]:
print("KNN Accuracies (Label Embedding Push)")
for k in [10, 20, 30, 50, 100, 200, 300, 500]:
    eval_knn(train_embeddings_modified_label_embeddings, train_labels, test_embeddings_norm, test_labels, k=k)

KNN Accuracies (Label Embedding Push)
Accuracy of kNN (k = 10): 0.6872
Accuracy of kNN (k = 20): 0.6753
Accuracy of kNN (k = 30): 0.6693
Accuracy of kNN (k = 50): 0.6664
Accuracy of kNN (k = 100): 0.6593
Accuracy of kNN (k = 200): 0.6501
Accuracy of kNN (k = 300): 0.6458
Accuracy of kNN (k = 500): 0.6407


In [9]:
print("KNN Accuracies (Label Centroid Push)")
for k in [10, 20, 30, 50, 100, 200, 300, 500]:
    eval_knn(train_embeddings_modified, train_labels, test_embeddings_norm, test_labels, k=k)

KNN Accuracies (Label Centroid Push)
Accuracy of kNN (k = 10): 0.7267
Accuracy of kNN (k = 20): 0.7238
Accuracy of kNN (k = 30): 0.7208
Accuracy of kNN (k = 50): 0.7139
Accuracy of kNN (k = 100): 0.7097
Accuracy of kNN (k = 200): 0.7053
Accuracy of kNN (k = 300): 0.7021
Accuracy of kNN (k = 500): 0.6952


In [10]:
print("KNN Accuracies (Attractor/Repellor Algo)")
for k in [10, 20, 30, 50, 100, 200, 300, 500]:
    eval_knn(refined_train_embeddings, train_labels, test_embeddings_norm, test_labels, k=k)

KNN Accuracies (Attractor/Repellor Algo)
Accuracy of kNN (k = 10): 0.7192
Accuracy of kNN (k = 20): 0.7220
Accuracy of kNN (k = 30): 0.7196
Accuracy of kNN (k = 50): 0.7170
Accuracy of kNN (k = 100): 0.7112
Accuracy of kNN (k = 200): 0.7033
Accuracy of kNN (k = 300): 0.7016
Accuracy of kNN (k = 500): 0.6949


In [11]:
print("KNN Accuracies (Attractor/Repeller Algo - Barycenter)")
for k in [10, 20, 30, 50, 100, 200, 300, 500]:
    eval_knn(refined_train_embeddings_bary, train_labels, test_embeddings_norm, test_labels, k=k)

KNN Accuracies (Attractor/Repeller Algo - Barycenter)
Accuracy of kNN (k = 10): 0.7039
Accuracy of kNN (k = 20): 0.7151
Accuracy of kNN (k = 30): 0.7131
Accuracy of kNN (k = 50): 0.7073
Accuracy of kNN (k = 100): 0.7028
Accuracy of kNN (k = 200): 0.7004
Accuracy of kNN (k = 300): 0.6936
Accuracy of kNN (k = 500): 0.6842


# PCA Reduction + Visualization

In [12]:
import numpy as np
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from sklearn.decomposition import PCA

def plot_embeddings(embeddings, labels):
    # PCA to reduce dimensions to 3
    pca = PCA(n_components=3)
    reduced_embeddings = pca.fit_transform(embeddings)

    single_labels = labels

    # Creating a color map with 20 unique colors
    colors = [f'rgba({r}, {g}, {b}, 0.8)' for r, g, b, _ in 255 * plt.cm.tab20(np.arange(20))]

    # Plotting using Plotly
    fig = go.Figure()

    for idx, label in enumerate(train_labels_text):
        # Select data points belonging to the current label
        indices = single_labels == idx
        fig.add_trace(go.Scatter3d(
            x=reduced_embeddings[indices, 0],
            y=reduced_embeddings[indices, 1],
            z=reduced_embeddings[indices, 2],
            mode='markers',
            marker=dict(
                size=5,
                color=colors[idx],  # Use specific color for each label
                opacity=0.8
            ),
            name=label  # Use label text for legend
        ))

    fig.update_layout(
        title='3D PCA Visualization of Word Embeddings',
        scene=dict(
            xaxis_title='PC1',
            yaxis_title='PC2',
            zaxis_title='PC3'
        ),
        legend_title="Labels",
        legend=dict(
            x=0,
            y=1,
            traceorder="normal",
            font=dict(
                family="sans-serif",
                size=12,
                color="black"
            ),
            bgcolor="LightSteelBlue",
            bordercolor="Black",
            borderwidth=2
        )
    )

    fig.show()

plot_embeddings(train_embeddings_norm, train_labels)
plot_embeddings(refined_train_embeddings, train_labels)
plot_embeddings(refined_train_embeddings_bary, train_labels)

# Average Pairwise Similarity Amongst Embeddings in the Same Class
Overall, the average pairwise similarity amongst embeddings in the train and test sets hovers around 30% (cosine similarity), which is surprisingly low.

In [13]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def average_pairwise_similarity(embeddings):
    # Compute the cosine similarity matrix
    similarity_matrix = cosine_similarity(embeddings)
    
    # Get the number of embeddings
    n = similarity_matrix.shape[0]
    sum_similarities = (np.sum(similarity_matrix) - n) / 2 # Get sum of similarities of unique pairs (exclude diagonal)
    num_pairs = ((n * (n - 1)) / 2) - n

    # Calculate average similarity
    average_similarity = sum_similarities / num_pairs
    
    return average_similarity

def compute_average_pairwise_similarities(_embeddings):
    pairwise_similarities = {}
    for _class in set(train_labels):
        _class_embeddings = _embeddings[train_labels == _class]

        avg_similarity = average_pairwise_similarity(_class_embeddings)
        pairwise_similarities[_class] = avg_similarity

        print(f"Average Pairwise Similarity in class {_class} (label: {train_labels_text[_class]}, size {len(_class_embeddings)}): {avg_similarity:.3f}")
    
    print("-" * 100)

    return pairwise_similarities

print("Base Similarities")
base_similarities = compute_average_pairwise_similarities(train_embeddings_norm)

print("Refined Similarities (Attractor/Repeller Algo)")
refined_similarities = compute_average_pairwise_similarities(refined_train_embeddings)

print("Refined Similarities (Centroid Label Based)")
refined_similarities = compute_average_pairwise_similarities(train_embeddings_modified)

print("Refined Similarities (Label Embedding Based)")
refined_similarities = compute_average_pairwise_similarities(train_embeddings_modified_label_embeddings)

print("Refined Similarities Barycenter")
bary_similarities = compute_average_pairwise_similarities(refined_train_embeddings_bary)

Base Similarities
Average Pairwise Similarity in class 0 (label: rec.autos, size 476): 0.332
Average Pairwise Similarity in class 1 (label: rec.sport.baseball, size 580): 0.334
Average Pairwise Similarity in class 2 (label: sci.space, size 573): 0.334
Average Pairwise Similarity in class 3 (label: sci.med, size 586): 0.349
Average Pairwise Similarity in class 4 (label: rec.motorcycles, size 577): 0.334
Average Pairwise Similarity in class 5 (label: comp.graphics, size 581): 0.363
Average Pairwise Similarity in class 6 (label: talk.politics.misc, size 584): 0.325
Average Pairwise Similarity in class 7 (label: soc.religion.christian, size 593): 0.303
Average Pairwise Similarity in class 8 (label: talk.politics.mideast, size 597): 0.310
Average Pairwise Similarity in class 9 (label: sci.crypt, size 597): 0.366
Average Pairwise Similarity in class 10 (label: misc.forsale, size 596): 0.379
Average Pairwise Similarity in class 11 (label: comp.sys.mac.hardware, size 587): 0.400
Average Pairwi

In [14]:
from sklearn.manifold import Isomap 

# Initialize Isomap
isomap = Isomap(n_components=100, n_neighbors=100, n_jobs=-1)

# Apply Isomap on the entire dataset before splitting into classes
train_embeddings_isomap = isomap.fit_transform(train_embeddings)
# test_embeddings_isomap = isomap.transform(test_embeddings)

iso_similarities = compute_average_pairwise_similarities(train_embeddings_isomap)

Average Pairwise Similarity in class 0 (label: rec.autos, size 476): 0.288
Average Pairwise Similarity in class 1 (label: rec.sport.baseball, size 580): 0.268
Average Pairwise Similarity in class 2 (label: sci.space, size 573): 0.297
Average Pairwise Similarity in class 3 (label: sci.med, size 586): 0.314
Average Pairwise Similarity in class 4 (label: rec.motorcycles, size 577): 0.295
Average Pairwise Similarity in class 5 (label: comp.graphics, size 581): 0.382
Average Pairwise Similarity in class 6 (label: talk.politics.misc, size 584): 0.300
Average Pairwise Similarity in class 7 (label: soc.religion.christian, size 593): 0.248
Average Pairwise Similarity in class 8 (label: talk.politics.mideast, size 597): 0.247
Average Pairwise Similarity in class 9 (label: sci.crypt, size 597): 0.392
Average Pairwise Similarity in class 10 (label: misc.forsale, size 596): 0.434
Average Pairwise Similarity in class 11 (label: comp.sys.mac.hardware, size 587): 0.333
Average Pairwise Similarity in c

# Baseline Dimensionality Reduction Methods
Using custom functions defined above. Re-run for empirical evaluation using existing library implementations due to long running-times.

In [15]:
from scipy.spatial.distance import cdist
from scipy.sparse.csgraph import shortest_path
from scipy.sparse.linalg import eigsh
from scipy.linalg import eigh

def _MDS_projection(_points, d):
    """
    _points: pairwise distsances
    """

    # Multi-dimensional scaling
    centering_matrix = np.eye(_points.shape[0]) - (np.ones(_points.shape) / _points.shape[0])
    gram_matrix = -0.5 * (centering_matrix @ (_points ** 2) @ centering_matrix)

    # Eigenvalue decomposition
    eigenvalues, eigenvectors = np.linalg.eigh(gram_matrix)

    # Get d largest eigenvalues/vectors
    top_d_eigenvalues = eigenvalues[-d:]
    top_d_eigenvectors = eigenvectors[:, -d:]

    # Project data
    Y = top_d_eigenvectors @ np.sqrt(np.diag(top_d_eigenvalues))

    return Y

def MDS(_points, d=2):
    pairwise_dists = cdist(_points, _points, metric='euclidean')

    return _MDS_projection(pairwise_dists, d=d)

def isomap(_points, n_neighbors=None, epsilon=None, d=2, method='dijkstra', verbose=False):
    N = _points.shape[0]

    # Construct neighborhood graph
    neighborhood_graph = cdist(_points, _points, metric='euclidean')

    if n_neighbors is not None:
        k_neighbors = np.argsort(neighborhood_graph, axis=1)[:, 1:n_neighbors + 1]
        for i in range(N):
            neighborhood_graph[i, ~np.isin(range(N), k_neighbors[i])] = np.inf
        
    if epsilon is not None:
        neighborhood_graph[neighborhood_graph > epsilon] = np.inf

    shortest_paths = neighborhood_graph.copy()
    if method == 'floyd':
        # Floyd's Algorithm
        for k in range(N):
            for i in range(N):
                for j in range(N):
                    if shortest_paths[i, k] + shortest_paths[k, j] < shortest_paths[i, j]:
                        shortest_paths[i, j] = shortest_paths[i, k] + shortest_paths[k, j]
    elif method == 'dijkstra':
        # Dijkstra
        shortest_paths = shortest_path(neighborhood_graph, method='D', directed=False)

    if verbose:
        print(shortest_paths)
        
    return _MDS_projection(shortest_paths, d=d)   

def LLE(_points, n_neighbors=5, d=2):
    N, D = _points.shape
    
    # Step 1: Find the k-nearest neighbors
    pairwise_distances = cdist(_points, _points, 'euclidean')
    k_neighbors_graph = np.argsort(pairwise_distances, axis=1)[:, 1:n_neighbors+1]  # Exclude self
    
    # Step 2: Compute the reconstruction weights
    W = np.zeros((N, N))
    for i in range(N):
        Z = _points[k_neighbors_graph[i]] - _points[i]
        
        # Covariance matrix from the differences
        C = np.dot(Z, Z.T)
        
        # Regularization to avoid singularity
        C += np.eye(n_neighbors) * 1e-3
        
        # Solve for reconstruction weights
        w = np.linalg.solve(C, np.ones(n_neighbors))
        w /= np.sum(w)  # Ensure weights sum to 1
        
        W[i, k_neighbors_graph[i]] = w
    
    # Step 3: Construct matrix M and solve for embeddings
    M = (np.eye(N) - W).T @ (np.eye(N) - W)
    eigenvalues, eigenvectors = np.linalg.eigh(M)
    
    # Skip the first eigenvalue and eigenvector (corresponding to the smallest eigenvalue, which is zero)
    return eigenvectors[:, 1:d+1]


def create_adjacency_matrix(X, method='gaussian', sigma=1.0, n_neighbors=None, epsilon=None):
    pairwise_distances = cdist(X, X, 'euclidean')
    W = np.zeros_like(pairwise_distances)

    if epsilon is not None:
        if method == 'gaussian':
            W = np.exp(-pairwise_distances ** 2 / (2 * sigma ** 2)) * (pairwise_distances < epsilon)
        else: 
            W = (pairwise_distances < epsilon).astype(float)
    elif n_neighbors is not None:
        for i in range(X.shape[0]):
            # Sort distances and select the indices of the n_neighbors nearest neighbors
            sorted_indices = np.argsort(pairwise_distances[i])[1:n_neighbors+1]
            if method == 'gaussian':
                for j in sorted_indices:
                    W[i, j] = np.exp(-pairwise_distances[i, j] ** 2 / (2 * sigma ** 2))
                    W[j, i] = W[i, j]  # Ensure symmetry
            else:
                W[i, sorted_indices] = 1
                W[sorted_indices, i] = 1  # Ensure symmetry

    return W

def create_graph_laplacian(W):
    D = np.diag(W.sum(axis=1))
    L = D - W
    return L, D

def laplacian_eigenmaps(_points, n_neighbors=None, epsilon=None, method='gaussian', sigma=1.0, d=2):
    W = create_adjacency_matrix(_points, method=method, sigma=sigma, n_neighbors=n_neighbors, epsilon=epsilon)
    L, D = create_graph_laplacian(W)

    eigenvalues, eigenvectors = eigh(L, D, subset_by_index=[1, d + 1])

    return eigenvectors

In [16]:
from joblib import Parallel, delayed
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.manifold import Isomap, LocallyLinearEmbedding, SpectralEmbedding
from sklearn.decomposition import PCA
from tqdm import tqdm

def train_and_test(X_train, y_train, X_test, y_test, dim_reduction_method='isomap', n_neighbors=5, n_components=2, k=10):
    if dim_reduction_method == 'isomap':
        model = Isomap(n_neighbors=n_neighbors, n_components=n_components)
    elif dim_reduction_method == 'pca':
        model = PCA(n_components=n_components)
    elif dim_reduction_method == 'lle':
        model = LocallyLinearEmbedding(n_neighbors=n_neighbors, n_components=n_components)
    elif dim_reduction_method == 'laplacian_eigenmaps':
        model = SpectralEmbedding(n_components=n_components, n_neighbors=n_neighbors)
    else:
        model = None

    X_train_transformed = model.fit_transform(X_train)
    X_test_transformed = model.transform(X_test)

    classifier = KNeighborsClassifier(n_neighbors=k, metric='cosine')
    classifier.fit(X_train_transformed, y_train)
    y_pred = classifier.predict(X_test_transformed)
    accuracy = accuracy_score(y_test, y_pred)
    
    return f"Accuracy with {dim_reduction_method} at {n_components} components: {accuracy:.2f}"

# Parallel execution
results = Parallel(n_jobs=-1)(delayed(train_and_test)(train_embeddings, train_labels, test_embeddings, test_labels, dim_reduction_method=method, n_components=i, k=20)
                               for i in tqdm([25, 50, 100, 200, 300, 400, 500, 600, 700])
                               for method in ['isomap', 'pca', 'lle'])

for result in results:
    print(result)

100%|██████████| 9/9 [00:02<00:00,  3.21it/s]


Accuracy with isomap at 25 components: 0.67
Accuracy with pca at 25 components: 0.69
Accuracy with lle at 25 components: 0.63
Accuracy with isomap at 50 components: 0.67
Accuracy with pca at 50 components: 0.71
Accuracy with lle at 50 components: 0.65
Accuracy with isomap at 100 components: 0.67
Accuracy with pca at 100 components: 0.72
Accuracy with lle at 100 components: 0.67
Accuracy with isomap at 200 components: 0.68
Accuracy with pca at 200 components: 0.72
Accuracy with lle at 200 components: 0.67
Accuracy with isomap at 300 components: 0.68
Accuracy with pca at 300 components: 0.72
Accuracy with lle at 300 components: 0.67
Accuracy with isomap at 400 components: 0.68
Accuracy with pca at 400 components: 0.72
Accuracy with lle at 400 components: 0.67
Accuracy with isomap at 500 components: 0.68
Accuracy with pca at 500 components: 0.72
Accuracy with lle at 500 components: 0.68
Accuracy with isomap at 600 components: 0.68
Accuracy with pca at 600 components: 0.72
Accuracy with ll

# Neighborhood Preserving Embeddings (NPE)
Implementation of NPE and its variations (supervised NPE and kernel NPE). 

In supervised NPE we use the class labels to inform a better weight matrix (i.e. adding greater weight to neighbors of the same class label). 

Kernel NPE, as expected, takes much longer to run.

In [17]:
class NPE:
    def __init__(self, n_components, n_neighbors):
        self.n_components = n_components
        self.n_neighbors = n_neighbors
        self.transformation_matrix = None
    
    def p1solver(self, data, x, neighbors, k):
        Z = data[neighbors].T
        Z = Z - np.repeat(x.T, len(neighbors), axis=0).reshape(data.shape[1], len(neighbors))
        C = Z.T @ Z
        C = C + (np.eye(C.shape[0]) * 1e-5)
        w = np.linalg.solve(C, np.ones(len(neighbors)))
        w_final = np.zeros(len(data))
        w_final[neighbors] = w
        return w_final / np.sum(w_final)
    
    def knn(self, k, data, test):
        return np.argsort(np.sum(data**2, axis=1) - 2 * test.dot(data.T), axis=0)[1:k+1]
    
    def NPEsolver(self, X, W, k):
        I = np.identity(X.shape[1])
        M = (I-W).T @ (I-W)
        T1 = np.linalg.inv(X @ X.T)
        T2 = X @ M @ X.T
        evalues, evectors = np.linalg.eigh(T1 @ T2)
        Y = evectors[:, 1:k+1]
        return Y
    
    def fit(self, X):
        W = []
        for i in range(len(X)):
            neighbors = self.knn(self.n_neighbors, X, X[i])
            W.append(self.p1solver(X, X[i], neighbors, self.n_components))
        W = np.asarray(W)
        self.transformation_matrix = self.NPEsolver(X.T, W, self.n_components)
    
    def transform(self, X):
        return (self.transformation_matrix.T @ X.T).T

# Usage
npe = NPE(n_components=300, n_neighbors=100)
npe.fit(train_embeddings_norm)

transformed_train_embeddings = npe.transform(train_embeddings_norm)
transformed_test_embeddings = npe.transform(test_embeddings_norm)

for k in [10, 20, 30, 50, 100, 200, 300, 500]:
    eval_knn(transformed_train_embeddings, train_labels, transformed_test_embeddings, test_labels, k=k)

Accuracy of kNN (k = 10): 0.7166
Accuracy of kNN (k = 20): 0.7222
Accuracy of kNN (k = 30): 0.7154
Accuracy of kNN (k = 50): 0.7115
Accuracy of kNN (k = 100): 0.7087
Accuracy of kNN (k = 200): 0.6969
Accuracy of kNN (k = 300): 0.6913
Accuracy of kNN (k = 500): 0.6774


In [18]:
class SupervisedNPE:
    def __init__(self, n_components, n_neighbors):
        self.n_components = n_components
        self.n_neighbors = n_neighbors
        self.transformation_matrix = None
    
    def p1solver(self, data, x, neighbors, label, neighbor_labels, k):
        Z = data[neighbors].T
        Z = Z - np.repeat(x.T, len(neighbors), axis=0).reshape(data.shape[1], len(neighbors))
        C = Z.T @ Z
        C = C + (np.eye(C.shape[0]) * 1e-5)
        
        # Assign higher weights to neighbors with the same class label
        w = np.zeros(len(neighbors))
        for i, neighbor_label in enumerate(neighbor_labels):
            if neighbor_label == label:
                w[i] = 1.0
            else:
                w[i] = 0.1
        
        w /= np.sum(w)
        w_final = np.zeros(len(data))
        w_final[neighbors] = w
        return w_final
    
    def knn(self, k, data, test):
        return np.argsort(np.sum(data**2, axis=1) - 2 * test.dot(data.T), axis=0)[1:k+1]
    
    def NPEsolver(self, X, W, k):
        I = np.identity(X.shape[1])
        M = (I-W).T @ (I-W)
        T1 = np.linalg.inv(X @ X.T)
        T2 = X @ M @ X.T
        evalues, evectors = np.linalg.eigh(T1 @ T2)
        Y = evectors[:, 1:k+1]
        return Y
    
    def fit(self, X, labels):
        W = []
        for i in range(len(X)):
            neighbors = self.knn(self.n_neighbors, X, X[i])
            neighbor_labels = labels[neighbors]
            W.append(self.p1solver(X, X[i], neighbors, labels[i], neighbor_labels, self.n_components))
        W = np.asarray(W)
        self.transformation_matrix = self.NPEsolver(X.T, W, self.n_components)
    
    def transform(self, X):
        return (self.transformation_matrix.T @ X.T).T

# Usage
snpe = SupervisedNPE(n_components=300, n_neighbors=100)
snpe.fit(train_embeddings_norm, train_labels)

transformed_train_embeddings = snpe.transform(train_embeddings_norm)
transformed_test_embeddings = snpe.transform(test_embeddings_norm)

for k in [10, 20, 30, 50, 100, 200, 300, 500]:
    eval_knn(transformed_train_embeddings, train_labels, transformed_test_embeddings, test_labels, k=k)

Accuracy of kNN (k = 10): 0.7239
Accuracy of kNN (k = 20): 0.7219
Accuracy of kNN (k = 30): 0.7214
Accuracy of kNN (k = 50): 0.7187
Accuracy of kNN (k = 100): 0.7131
Accuracy of kNN (k = 200): 0.7011
Accuracy of kNN (k = 300): 0.6957
Accuracy of kNN (k = 500): 0.6840


In [19]:
from sklearn.decomposition import KernelPCA

print("Computing KPCA...")
kpca = KernelPCA(n_components=300, kernel="rbf", gamma=0.01)
train_embeddings_kpca = kpca.fit_transform(train_embeddings_norm)
test_embeddings_kpca = kpca.transform(test_embeddings_norm)

print("Running NPE on KPCA data...")
npe = NPE(n_components=300, n_neighbors=100)
npe.fit(train_embeddings_kpca)

transformed_train_embeddings = npe.transform(train_embeddings_kpca)
transformed_test_embeddings = npe.transform(test_embeddings_kpca)


Computing KPCA...
Running NPE on KPCA data...


In [20]:
for k in [10, 20, 30, 50, 100, 200, 300, 500]:
    eval_knn(transformed_train_embeddings, train_labels, transformed_test_embeddings, test_labels, k=k)

Accuracy of kNN (k = 10): 0.7230
Accuracy of kNN (k = 20): 0.7228
Accuracy of kNN (k = 30): 0.7174
Accuracy of kNN (k = 50): 0.7127
Accuracy of kNN (k = 100): 0.7081
Accuracy of kNN (k = 200): 0.7027
Accuracy of kNN (k = 300): 0.6944
Accuracy of kNN (k = 500): 0.6806


In [21]:
print("Running Supervised Kernel NPE on KPCA data...")

npe = SupervisedNPE(n_components=300, n_neighbors=100)
npe.fit(train_embeddings_kpca, train_labels)

transformed_train_embeddings = npe.transform(train_embeddings_kpca)
transformed_test_embeddings = npe.transform(test_embeddings_kpca)

print("Supervised Kernel NPE Accuracies")
for k in [10, 20, 30, 50, 100, 200, 300, 500]:
    eval_knn(transformed_train_embeddings, train_labels, transformed_test_embeddings, test_labels, k=k)

Running Supervised Kernel NPE on KPCA data...
Supervised Kernel NPE Accuracies
Accuracy of kNN (k = 10): 0.7244
Accuracy of kNN (k = 20): 0.7254
Accuracy of kNN (k = 30): 0.7195
Accuracy of kNN (k = 50): 0.7150
Accuracy of kNN (k = 100): 0.7088
Accuracy of kNN (k = 200): 0.7039
Accuracy of kNN (k = 300): 0.6961
Accuracy of kNN (k = 500): 0.6836
